## Modules

In [1]:
import torch
import os
import pandas as pd
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    GenerationConfig
)
from peft.tuners.lora import LoraLayer

from trl import SFTTrainer

## Config

In [2]:
TESTING = False
os.environ["WANDB_DISABLED"] = "true"
MAX_SEQ_LENGTH = None
OUTPUT_DIR = "results/results_w_eos_13b"

## Prepare trainer and tokenizer

In [3]:
model_path = "/mnt/shared/tibor/Llama-2-13b-chat-hf"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True, add_eos_token=True)
#Create a new token and add it to the tokenizer
tokenizer.add_special_tokens({"pad_token":"<pad>"})
tokenizer.padding_side = 'left'
print(f"{tokenizer.model_max_length=}")

tokenizer.model_max_length=1000000000000000019884624838656


In [4]:
print(f"{tokenizer.eos_token_id=}")
inputs = tokenizer("Ez egy teszt", return_tensors="pt")
input_ids = inputs["input_ids"].cuda()
input_ids

tokenizer.eos_token_id=2


tensor([[    1, 18817,  5524,   260, 23293,     2]], device='cuda:0')

In [5]:
# compute_dtype = getattr(torch, "float16")
compute_dtype = "bfloat16"

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_path, quantization_config=bnb_config, device_map={"": 0}
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
#Resize the embeddings
model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=16) # https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc

Embedding(32016, 5120)

In [7]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False # Gradient checkpointing is used by default but not compatible with caching

In [8]:
model = prepare_model_for_kbit_training(model) # ?
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["q_proj","v_proj"]
)

In [9]:

training_arguments=None
if TESTING:
    training_arguments = TrainingArguments(
        output_dir=OUTPUT_DIR,
        evaluation_strategy="steps",
        do_eval=True,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        per_device_eval_batch_size=4,
        log_level="debug",
        optim="paged_adamw_32bit",
        save_steps=2, #change to 500, test: 2
        logging_steps=1, #change to 100, test: 1
        learning_rate=1e-4,
        eval_steps=5, #change to 200, test: 5
        bf16=True,
        max_grad_norm=0.3,
        # num_train_epochs=3, # remove "#"
        max_steps=10, #remove this
        warmup_ratio=0.03,
        lr_scheduler_type="constant",
)
else:
    training_arguments = TrainingArguments(
            output_dir=OUTPUT_DIR,
            evaluation_strategy="steps",
            do_eval=True,
            per_device_train_batch_size=4,
            gradient_accumulation_steps=1,
            per_device_eval_batch_size=4,
            log_level="debug",
            optim="paged_adamw_32bit",
            save_steps=500, #change to 500, test: 2
            logging_steps=100, #change to 100, test: 1
            learning_rate=1e-4,
            eval_steps=200, #change to 200, test: 5
            bf16=True,
            max_grad_norm=0.3,
            num_train_epochs=3, # remove "#"
            # max_steps=10, #remove this
            warmup_ratio=0.03,
            lr_scheduler_type="constant",
            report_to=None
    )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [10]:
dataset = load_dataset("csv", 
                       data_files={'train': 'data/train.csv', 'eval': 'data/eval.csv'},
                       delimiter=";", 
                       column_names=['question', 'context', 'answer', 'text'])

### Check for longest input

In [11]:
inputs = tokenizer("Ez egy teszt", return_tensors="pt")
input_ids = inputs["input_ids"].cuda()
print(f"{input_ids=}")


input_ids=tensor([[    1, 18817,  5524,   260, 23293,     2]], device='cuda:0')


In [12]:
train_df = pd.read_csv("data/train.csv", sep=';')
eval_df = pd.read_csv("data/test.csv", sep=';')
test_df = pd.read_csv("data/test.csv", sep=';')

all_data=[train_df, eval_df, test_df]
longest_embedding=0


for df in all_data:
    for index, row in df.loc[:].iterrows():

        inputs = tokenizer(row["text"], return_tensors="pt")
        input_ids = inputs["input_ids"].cuda()
        
        if len(input_ids[0]) > longest_embedding:
            longest_embedding = len(input_ids[0])

    print(f"{longest_embedding=}")
    longest_embedding=0
        

longest_embedding=985
longest_embedding=965
longest_embedding=965


## Train

In [13]:
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset['train'],
        eval_dataset=dataset['eval'],
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=min(tokenizer.model_max_length, 1024),  # default: min(tokenizer.model_max_length, 1024),
        tokenizer=tokenizer,
        args=training_arguments,
)

/mnt/shared/tibor/miniconda3/envs/test/lib/python3.10/site-packages/peft/utils/other.py:133: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/2253 [00:00<?, ? examples/s]

Map:   0%|          | 0/484 [00:00<?, ? examples/s]

/mnt/shared/tibor/miniconda3/envs/test/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
The model is quantized. To train this model you need to add additional modules inside the model such as adapters using `peft` library and freeze the model weights. Please check the examples in https://github.com/huggingface/peft for more details.
[codecarbon INFO @ 17:05:51] [setup] RAM Tracking...
[codecarbon INFO @ 17:05:51] [setup] GPU Tracking...
[codecarbon INFO @ 17:05:51] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:05:51] [setup] CPU Tracking...
[codecarbon WARNING @ 17:05:51] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 17:05:53] We saw tha

In [14]:
trainer.train()

Currently training with a batch size of: 4
***** Running training *****
  Num examples = 2,253
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1,692
  Number of trainable parameters = 6,553,600
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
200,1.257400,1.253873
400,1.238400,1.216970
600,1.170600,1.193339
800,1.140600,1.176519
1000,1.127900,1.157329
1200,1.064900,1.140926
1400,1.024600,1.124174
1600,1.025500,1.103783


[codecarbon INFO @ 17:06:10] Energy consumed for RAM : 0.000049 kWh. RAM Power : 11.757657051086426 W
[codecarbon INFO @ 17:06:10] Energy consumed for all GPUs : 0.000000 kWh. Total GPU Power : 0.0 W
[codecarbon INFO @ 17:06:10] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:06:10] 0.000226 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:06:25] Energy consumed for RAM : 0.000098 kWh. RAM Power : 11.757657051086426 W
[codecarbon INFO @ 17:06:25] Energy consumed for all GPUs : 0.000000 kWh. Total GPU Power : 0.0 W
[codecarbon INFO @ 17:06:25] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:06:25] 0.000452 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:06:40] Energy consumed for RAM : 0.000147 kWh. RAM Power : 11.757657051086426 W
[codecarbon INFO @ 17:06:40] Energy consumed for all GPUs : 0.000000 kWh. Total GPU Power : 0.0 W
[codecarbon INFO @ 17:06:40] Energy 

TrainOutput(global_step=1692, training_loss=1.1400326713038957, metrics={'train_runtime': 5002.1036, 'train_samples_per_second': 1.351, 'train_steps_per_second': 0.338, 'total_flos': 1.624000516061184e+17, 'train_loss': 1.1400326713038957, 'epoch': 3.0})